In [2]:
import os
import pickle
import torch
import numpy as np
from pyfaidx import Fasta
import matplotlib.pyplot as plt
from total_rnaseq_evaluator import SpliceSiteEvaluator

In [3]:
# Paths to your files
BAM_FILE = "/data/Splice/data/ENCFF800HIP.bam"
FASTA_FILE = "/data/genomes/hg38/hg38.fa"

# Initialize evaluator
evaluator = SpliceSiteEvaluator(
    bam_file=BAM_FILE, 
    fasta_file=FASTA_FILE
)

Exception ignored in: <function Faidx.__del__ at 0x14a0cfebd630>
Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/pyfaidx/__init__.py", line 883, in __del__
    self.__exit__()
  File "/opt/venv/lib/python3.10/site-packages/pyfaidx/__init__.py", line 889, in __exit__
    self.file.close()
AttributeError: 'Faidx' object has no attribute 'file'


Please ignore this warning if you are using hg38
hg19 fasta not found


/home/christian.ramirez1-umw/data/Splice/SpliceTransformer/tasks_annotate_mutations.py:411: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  save_dict = torch.load(


torch device:cuda


In [4]:
# Generate ground truth
acceptor_ground_truth, donor_ground_truth = evaluator.generate_ground_truth()

Found existing junctions.bed, using existing file...
Median junction read count: 8.0

Per-chromosome junction counts:
chr1: Total junctions 10531, Positive acceptor sites: 5249, Positive donor sites: 5176
chr3: Total junctions 6034, Positive acceptor sites: 2882, Positive donor sites: 2861
chr5: Total junctions 4378, Positive acceptor sites: 2095, Positive donor sites: 2077
chr7: Total junctions 4981, Positive acceptor sites: 2218, Positive donor sites: 2194
chr9: Total junctions 4182, Positive acceptor sites: 2052, Positive donor sites: 2027


In [ ]:
# Generate predictions
acceptor_predictions, donor_predictions = evaluator.generate_predictions(
    ground_truth=acceptor_ground_truth
)

In [5]:
def save(data, name):
    """Save object to a pickle file in current directory."""
    with open(f"{name}.pkl", 'wb') as f:
        pickle.dump(data, f)

def load(name):
    """Load object from a pickle file in current directory."""
    with open(f"{name}.pkl", 'rb') as f:
        return pickle.load(f)

In [6]:
# Save objects
save(acceptor_ground_truth, "acceptor_ground_truth")
save(donor_ground_truth, "donor_ground_truth")
# save(acceptor_predictions, "acceptor_predictions")
# save(donor_predictions, "donor_predictions")

# Load objects
acceptor_ground_truth = load("acceptor_ground_truth")
donor_ground_truth = load("donor_ground_truth")
acceptor_predictions = load("acceptor_predictions")
donor_predictions = load("donor_predictions_corrected")

In [7]:
# chromosomes = ['chr1', 'chr3', 'chr5', 'chr7', 'chr9']
chromosomes = ['chr1']

acceptor_prediction_indices = np.concatenate([
    np.where(acceptor_predictions[chrom] >= 0.90)[0] 
    for chrom in chromosomes
])
print(acceptor_prediction_indices[:20])
print(len(acceptor_prediction_indices))
acceptor_truth_indices = np.concatenate([
    np.where(acceptor_ground_truth[chrom] == 1)[0] 
    for chrom in chromosomes
])
print(acceptor_truth_indices[:20])
print(len(acceptor_truth_indices))

donor_prediction_indices = np.concatenate([
    np.where(donor_predictions[chrom] >= 0.90)[0] 
    for chrom in chromosomes
])
print(donor_prediction_indices[:20])
print(len(donor_prediction_indices))
donor_truth_indices = np.concatenate([
    np.where(donor_ground_truth[chrom] == 1)[0] 
    for chrom in chromosomes
])
print(donor_truth_indices[:20])
print(len(donor_truth_indices))

print(len(set(acceptor_prediction_indices) & set(acceptor_truth_indices)))
print(len(set(donor_prediction_indices) & set(donor_truth_indices)))

print(donor_predictions)

# print(acceptor_ground_truth['chr1'][12612+4000-5:12612+4000+5])

[ 27573  62915 198095 829002 925921 930154 931038 935771 939039 939274
 942409 942558 943252 943697 943907 948510 961292 961628 961825 962285]
10999
[ 15038  15877  16952  17328  17704  18009  18364  24808  24819 139847
 155788 164334 165942 168178 169139 172645 173850 185580 186415 187214]
5249
[ 30667 201181 366259 779092 827775 829104 882367 924948 926013 930336
 931089 935896 939129 939412 942251 942488 943058 943377 943808 960800]
10685
[ 14744  14969  16673  16957  17286  17643  17962  18272  18294 139632
 146417 158622 164695 165883 168099 169174 172599 185251 185490 186370]
5176
1
0
{'chr1': array([0.00000000e+00, 1.82558438e-07, 1.78821651e-07, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 'chr3': array([0.00000000e+00, 1.82558438e-07, 1.78821651e-07, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 'chr5': array([0.00000000e+00, 1.82558438e-07, 1.78821651e-07, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 'chr7': array([0.00000000e

In [ ]:
# Calculate AUPRC
acceptor_auprc = evaluator.calculate_auprc(acceptor_ground_truth, acceptor_predictions)
donor_auprc = evaluator.calculate_auprc(donor_ground_truth, donor_predictions)
mean_auprc = (acceptor_auprc + donor_auprc) / 2
print(acceptor_auprc)
print(donor_auprc)
print(f"{mean_auprc:.4f}")

In [ ]:
# def diagnose_splice_site_alignment(acceptor_ground_truth, donor_ground_truth, 
#                                     acceptor_predictions, donor_predictions, 
#                                     threshold=0.9):
#     """
#     Diagnostic method to analyze alignment between ground truth and predictions
    
#     Args:
#         acceptor_ground_truth (dict): Ground truth for acceptor sites
#         donor_ground_truth (dict): Ground truth for donor sites
#         acceptor_predictions (dict): Predictions for acceptor sites
#         donor_predictions (dict): Predictions for donor sites
#         threshold (float): Probability threshold for considering a prediction positive
#     """
#     print("Splice Site Alignment Diagnostics:")
    
#     # Collect ground truth positions for each chromosome
#     ground_truth_stats = {}
    
#     for chrom in acceptor_ground_truth.keys():
#         # Find ground truth positions
#         acceptor_gt_pos = np.where(acceptor_ground_truth[chrom] == 1)[0]
#         donor_gt_pos = np.where(donor_ground_truth[chrom] == 1)[0]
        
#         # Find predicted positions
#         acceptor_pred_pos = np.where(acceptor_predictions[chrom] >= threshold)[0]
#         donor_pred_pos = np.where(donor_predictions[chrom] >= threshold)[0]
        
#         # Calculate matches
#         acceptor_matches = np.intersect1d(acceptor_gt_pos, acceptor_pred_pos)
#         donor_matches = np.intersect1d(donor_gt_pos, donor_pred_pos)
        
#         # Store statistics
#         ground_truth_stats[chrom] = {
#             'total_acceptor_gt': len(acceptor_gt_pos),
#             'total_donor_gt': len(donor_gt_pos),
#             'total_acceptor_pred': len(acceptor_pred_pos),
#             'total_donor_pred': len(donor_pred_pos),
#             'acceptor_matches': len(acceptor_matches),
#             'donor_matches': len(donor_matches),
#             'acceptor_match_percentage': len(acceptor_matches) / len(acceptor_gt_pos) * 100 if acceptor_gt_pos.size > 0 else 0,
#             'donor_match_percentage': len(donor_matches) / len(donor_gt_pos) * 100 if donor_gt_pos.size > 0 else 0
#         }
    
#     # Print detailed statistics
#     print("\nDetailed Chromosome Statistics:")
#     for chrom, stats in ground_truth_stats.items():
#         print(f"\n{chrom} Statistics:")
#         print(f"Acceptor Sites - Ground Truth: {stats['total_acceptor_gt']}, "
#               f"Predictions: {stats['total_acceptor_pred']}, "
#               f"Matches: {stats['acceptor_matches']} "
#               f"(Match %: {stats['acceptor_match_percentage']:.2f}%)")
#         print(f"Donor Sites - Ground Truth: {stats['total_donor_gt']}, "
#               f"Predictions: {stats['total_donor_pred']}, "
#               f"Matches: {stats['donor_matches']} "
#               f"(Match %: {stats['donor_match_percentage']:.2f}%)")
    
#     # Compute overall statistics
#     total_stats = {
#         'total_acceptor_gt': sum(stats['total_acceptor_gt'] for stats in ground_truth_stats.values()),
#         'total_donor_gt': sum(stats['total_donor_gt'] for stats in ground_truth_stats.values()),
#         'total_acceptor_pred': sum(stats['total_acceptor_pred'] for stats in ground_truth_stats.values()),
#         'total_donor_pred': sum(stats['total_donor_pred'] for stats in ground_truth_stats.values()),
#         'total_acceptor_matches': sum(stats['acceptor_matches'] for stats in ground_truth_stats.values()),
#         'total_donor_matches': sum(stats['donor_matches'] for stats in ground_truth_stats.values())
#     }
    
#     print("\nOverall Statistics:")
#     print(f"Total Acceptor Ground Truth Sites: {total_stats['total_acceptor_gt']}")
#     print(f"Total Acceptor Predicted Sites: {total_stats['total_acceptor_pred']}")
#     print(f"Total Acceptor Matches: {total_stats['total_acceptor_matches']}")
    
#     print(f"\nTotal Donor Ground Truth Sites: {total_stats['total_donor_gt']}")
#     print(f"Total Donor Predicted Sites: {total_stats['total_donor_pred']}")
#     print(f"Total Donor Matches: {total_stats['total_donor_matches']}")

In [ ]:
# diagnose_splice_site_alignment(
#     acceptor_ground_truth, 
#     donor_ground_truth, 
#     acceptor_predictions, 
#     donor_predictions
# )

In [ ]:
# Load predictions
with open('acceptor_predictions.pkl', 'rb') as f:
    donor_pred = pickle.load(f)

# Create new dictionary for corrected predictions
corrected = {}

# Shift each chromosome's predictions
for chrom in donor_pred:
    original = donor_pred[chrom]  # Get array for chromosome
    shifted = np.zeros_like(original)  # Create zero array same size
    shifted[1:] = original[:-1]  # Shift values right by 1
    corrected[chrom] = shifted

# Save corrected predictions
with open('donor_predictions_corrected.pkl', 'wb') as f:
    pickle.dump(corrected, f)